In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import cv2
import os
import ast
import random
import shutil
from PIL import Image
import glob
import matplotlib.pyplot as plt
from IPython.display import display
import matplotlib.patches as patches

In [3]:
def load_images_from_folder(folder_path):
    img_dict = {}
    filenames = sorted(os.listdir(folder_path))

    for filename in filenames:
        image_path = os.path.join(folder_path, filename)
        img_dict[filename] = image_path

    return img_dict

In [4]:
def assign_class(label):
    if label == "Frackels":
        return "1"
    elif label == "Acne":
        return "0"
    elif label == "Acne scars":
        return "2"
    else:
        return '10'

In [5]:
def extract_annots_val_df(df, file_name):
    anno = []

    df = df[df["filename"] == file_name]

    for index, row in df.iterrows():
        data = {}

        data["class"] =  int(assign_class(label=row['class']))  #row['class']
        data["bbox_h"] = row['bbox_height']
        data["bbox_w"] = row['bbox_width']

        #Converting Center coordinates to bottom left and right coordinates JUST for PLOTTING
        data["cx"] = row['x_center'] - (row['bbox_width'] / 2)
        data["cy"] = row['y_center'] - (row['bbox_height'] / 2)


        data['is_pred'] = False

        anno.append(data)
    return anno

In [6]:
def txt_file_annots(file_path):
    anno = []

    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            try:
                bbox = {}
                line = line.strip().split(' ')
                cls = int(line[0]) #assign_class(label=row['class'])
                x = float(line[1])
                y = float(line[2])
                w = float(line[3])
                h = float(line[4])
                conf = float(line[5])

                #Converting Center coordinates to bottom left and right coordinates JUST for PLOTTING
                x = x - (w / 2)
                y = y - (h / 2)

                bbox = {'class': cls,'cx': x, 'cy': y, 'bbox_w': w, 'bbox_h': h, 'conf': conf, 'is_pred': True}
                anno.append(bbox)

            except ValueError as e:
                print(f"Skipping line: {line} due to error: {str(e)}")

    return anno


In [7]:
def assign_color_pred(label):
    if label == 1:  ## Frackels
        return "cyan"
    elif label == 0:  ## Acne
        return "purple"
    elif label == 2:  ## Acne scars
      return "yellow"
    else:
        return "black"

In [8]:
def assign_color_actual(label):
    if label == 1:      #"Frackels":
        return "red"
    elif label == 0:    #"Acne":
        return "green"
    elif label == 2:    #"Acne scars":
        return "blue"
    # elif label == "Melasma":
    #     return ""
    # elif label == "Uneven Skin tone":
    #     return "white"
    # elif label == "Mole and Tags":
    #     return "purple"
    # elif label == "Wrinkels":
    #     return "cyan"
    # elif label == "Dark Circles":
    #     return "gray"
    # elif label == "Tear Trough":
    #     return "magenta"
    else:
        return "black"

In [9]:
def val_plot(image_path, annots, key):
    img = Image.open(image_path)
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))  #plt.subplots(figsize=(5, 5))
    ax[0].imshow(img)

    for i in annots:
      cls = i['class']
      x, y, h, w = i["cx"],i["cy"], i["bbox_h"], i["bbox_w"]
      img_w, img_h = img.width, img.height

      if i['is_pred'] == True:
        # Adding class label and confidence of bbox
        label = f"({i['conf']*100:.0f})"      #f"{cls} ({i['conf']:.2f})"
        # ax[0].text(x*img_w, y*img_h - 5, label, color=assign_color_pred(cls), fontsize=5)

        if i.get('is_correct') is not None:
          color = 'w'
        else:
          color = assign_color_pred(cls)
      else:
        color = assign_color_actual(cls)


      #Multipying with image width and height to DENORMALISE jut for plotting.
      rect = patches.Rectangle((x*img_w, y*img_h), w*img_w, h*img_h, linewidth=0.5, edgecolor=color, facecolor='none')
      ax[0].add_artist(rect)


    key = Image.open(key)
    ax[1].imshow(key)
    ax[1].axis('off')

    plt.show()

In [10]:
def calculate_iou(box1, box2):
    # Calculate intersection area
    x1 = max( box1['cx'] - (box1['bbox_w'] / 2), box2['cx'] - (box2['bbox_w'] / 2) )
    y1 = max( box1['cy'] - (box1['bbox_h'] / 2), box2['cy'] - (box2['bbox_h'] / 2) )
    x2 = min( box1['cx'] + (box1['bbox_w'] / 2), box2['cx'] + (box2['bbox_w'] / 2) )
    y2 = min( box1['cy'] + (box1['bbox_h'] / 2), box2['cy'] + (box2['bbox_h'] / 2) )

    intersection_area = max(0, x2 - x1) * max(0, y2 - y1)

    # Calculate union area
    box1_area = box1['bbox_w'] * box1['bbox_h']
    box2_area = box2['bbox_w'] * box2['bbox_h']
    union_area = box1_area + box2_area - intersection_area

    # Calculate IoU
    iou = intersection_area / union_area

    return iou

In [11]:
def non_max_suppression(predictions, threshold):
    # Sort predictions by confidence score in highest to lowest order
    sorted_predictions = sorted(predictions, key=lambda x: x['conf'], reverse=True)
    selected_predictions = []

    while len(sorted_predictions) > 0:
        best_prediction = sorted_predictions[0]
        selected_predictions.append(best_prediction)
        remaining_predictions = []

        for i in range(1, len(sorted_predictions)):
            current_prediction = sorted_predictions[i]
            iou = calculate_iou(best_prediction, current_prediction)

            if iou < threshold:
                remaining_predictions.append(current_prediction)

        sorted_predictions = remaining_predictions

    return selected_predictions

In [12]:
def non_max_suppression_class(predictions, threshold):
    # Create a dictionary to store selected predictions for each class
    selected_predictions = {}

    # Group predictions by class
    for prediction in predictions:
        cls = prediction['class']
        if cls not in selected_predictions:
            selected_predictions[cls] = []
        selected_predictions[cls].append(prediction)

    # Apply non-maximum suppression for each class
    for cls in selected_predictions.keys():
        class_predictions = selected_predictions[cls]

        # Sort predictions by confidence score in highest to lowest order
        sorted_predictions = sorted(class_predictions, key=lambda x: x['conf'], reverse=True)
        remaining_predictions = []

        while len(sorted_predictions) > 0:
            best_prediction = sorted_predictions[0]
            remaining_predictions.append(best_prediction)
            current_predictions = []

            for i in range(1, len(sorted_predictions)):
                current_prediction = sorted_predictions[i]
                iou = calculate_iou(best_prediction, current_prediction)

                if iou < threshold:
                    current_predictions.append(current_prediction)

            sorted_predictions = current_predictions

        selected_predictions[cls] = remaining_predictions

    # Flatten the selected predictions into a single list
    final_predictions = []
    for class_predictions in selected_predictions.values():
        final_predictions.extend(class_predictions)

    return final_predictions

In [13]:
def check_correct(predictions, actuals, threshold):
  correct_bboxes = []

  for gt_annot in actuals:
    for pred_annot in predictions:
      if gt_annot.get('class') == pred_annot.get('class'):
        iou = calculate_iou(gt_annot, pred_annot)

        if iou > threshold:
          pred_annot.update(is_correct = True)
          correct_bboxes.append(pred_annot)

  return correct_bboxes

In [14]:
def create_val_df(original_df, list_of_attr, val_labels):
    # Only including chosen classes
    df = original_df[original_df['class'].isin(list_of_attr)]

    # Extract the file extension from the filenames in the DataFrame
    df['file_extension'] = df['filename'].str.rsplit('.', 1).str[1]

    # Remove the extension from the filenames in val_labels
    filenames = [os.path.splitext(filename)[0] for filename in val_labels]

    # Filter the DataFrame based on the filenames without the extension
    val_df = df[df['filename'].str.rsplit('.', 1).str[0].isin(filenames)]

    return val_df

In [15]:
def assign_class_label(label):
    if label == 1:
        return "Frackels"
    elif label == 0:
        return "Acne"
    elif label == 2:
        return "Acne scars"
    else:
        return None

In [16]:
def print_results(total_actual,total_preds,total_correct, conf_thresh,iou_thresh,correct_iou_thresh, gt_comp,correct_comp,pred_comp):

  # for cls,count in gt_comp.items():
  # print(cls,count)
  print('\n>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>')
  print('>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>')
  print('\nconf_thresh =',conf_thresh)
  print("iou_thresh =",iou_thresh)
  print("correct_iou_thresh =",correct_iou_thresh)

  print('\nTotal Actual =', total_actual,'\t\tActual =', gt_comp)
  print("Total Preds =",total_preds,'\t\tPreds =', pred_comp)
  print("Total Correct =",total_correct,'\t\tCorrect =', correct_comp)

  correct_per = (total_correct*100) / total_preds
  wrong_per = ((total_preds - total_correct)*100) / total_preds

  print(f"\nCorrect = {correct_per:.2f}%")
  print(f"Wrong = {wrong_per:.2f}%")

In [17]:
def count_instances(a, dict_):
  # Create a dictionary to store class counts
  class_counts = {}

  # Iterate over the list of dictionaries
  for item in dict_:
      class_label = assign_class_label(item['class'])
      if class_label in class_counts:
          class_counts[class_label] += 1
      else:
          class_counts[class_label] = 1

  #Adding the current dict to the previous dicts to compile.
  res = {key: a.get(key, 0) + class_counts.get(key, 0) for key in set(a) | set(class_counts)}
  return res

In [18]:
df = pd.read_csv('/content/drive/MyDrive/Colab_Skin_Disease/1683inst_339imgs.csv')

In [19]:
# images = load_images_from_folder('/content/drive/MyDrive/Colab_Skin_Disease/via-2.0.12')
# For augmented
images = load_images_from_folder('/content/drive/MyDrive/Colab_Skin_Disease/YOLO_data/val/images')

In [20]:
key = '/content/drive/MyDrive/Colab_Skin_Disease/Drawio keys/multi_class_key.drawio.png'

!!CHANGE EXP FILE TO THE RECENT VAL FILE.>>>>>>>>>>>>>

In [21]:
val_lab_path = '/content/drive/MyDrive/Colab_Skin_Disease/yolov5/runs/val/exp49/labels/'

In [22]:
val_labels = os.listdir(val_lab_path)

In [23]:
# list_of_attr = ['Acne']
# val_df = create_val_df(df,list_of_attr,val_labels)
# For augmented
val_df = pd.read_csv('/content/drive/MyDrive/Colab_Skin_Disease/val_df.csv')

In [24]:
conf_thresh = 0.1
iou_thresh = 0.25 #Lower --> less boxes /// This means 2 boxes can share MAX 30% of their area, it discards the rest.
correct_iou_thresh = 0.01

total_actual = 0
total_preds = 0
total_correct = 0

gt_comp,pred_comp,correct_comp = {}, {}, {}

for name, path in images.items():

  label_id = name.split('.')[0] + '.txt'

  if label_id in val_labels:
    conf_pred_annots=[]
    n = os.path.splitext(name)

    pred_annots = txt_file_annots(val_lab_path + n[0] + ".txt")
    gt_annots = extract_annots_val_df(val_df,name)

    #Loop to apply cutoff of Confidence.
    for annot in pred_annots:
      if annot['conf'] > conf_thresh:
        conf_pred_annots.append(annot)

    #Applying NMS
    nms_pred_annots = non_max_suppression_class(predictions=conf_pred_annots, threshold=iou_thresh)

    #Checking correct predictions by comparing IoU with actuals
    correct_annots = check_correct(predictions=nms_pred_annots, actuals=gt_annots, threshold=correct_iou_thresh)

    #Removing the correct annots from the predictions to plot them only once.
    # print(gt_annots)
    # print(nms_pred_annots)
    # print(correct_annots)
    # nms_pred_annots = [{key: list1_dict[key] - list2_dict[key] for key in list1_dict} for list1_dict, list2_dict in zip(nms_pred_annots, correct_annots)]

    #Adding all preds, actuals and correct annots for plotting
    annots = nms_pred_annots + gt_annots + correct_annots

    gt_comp = count_instances(gt_comp, gt_annots)
    pred_comp = count_instances(pred_comp, nms_pred_annots)
    correct_comp = count_instances(correct_comp, correct_annots)

    val_plot(image_path=path, annots=annots, key=key)
    print(name, ' Actual=', len(gt_annots), 'Preds=',len(nms_pred_annots), ' Correct=', len(correct_annots))
    total_actual += len(gt_annots)
    total_preds += len(nms_pred_annots)
    total_correct += len(correct_annots)
    # break

print_results(total_actual, total_preds, total_correct, conf_thresh, iou_thresh, correct_iou_thresh,gt_comp, correct_comp,pred_comp)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
conf_thresh = 0.1
iou_thresh = 0.25
correct_iou_thresh = 0.01

Total Actual = 428 		Actual = {'Frackels': 237, 'Acne': 191}
Total Preds = 1020 		Preds = {'Frackels': 356, 'Acne': 664}
Total Correct = 147 		Correct = {'Frackels': 59, 'Acne': 88}

Correct = 14.41%
Wrong = 85.59%